# Accessing Dynamics365 and Outlook365 With Autogen

Prerequisites
- Access to Dynamics365 with Service Principles and appropriate access. Refer to [Microsoft AAD with Dynamics](https://learn.microsoft.com/en-us/power-apps/developer/data-platform/)walkthrough-register-app-azure-active-directory for connection details.
- Access to Outlook 365 with Service principle [Microsoft AAD with Outlook][Microsoft AAD with Dynamics](https://learn.microsoft.com/en-us/outlook/rest/get-started)
- Autogen : [installation guide](/docs/installation/). 
- Read through Task Decomposition for understanding how to work with function calls in Agentic Architecture

### Note : This is a sample approach and may not store the relevent details for Service Principles securily. Please follow your organization's guideline to handle secrets and enviroment variables. 

Author Prateek Dhiman Steve Vassallo 

Microsoft 

In [1]:
import time
import os 
from autogen import AssistantAgent, UserProxyAgent, config_list_from_json
from autogen.cache import Cache
from autogen.coding import DockerCommandLineCodeExecutor, LocalCommandLineCodeExecutor
from typing_extensions import Annotated
import autogen
from autogen.cache import Cache
from dotenv import load_dotenv

In [7]:
# Lets fetch the environment variables. Make sure you have a .env file in same directory as this file
# Load the .env file
load_dotenv('.env')
api_key = os.getenv('api_key')
api_base = os.getenv('api_base')
api_version = os.getenv('api_version')

### LLM Configuations 

In [8]:
# If the config list is small, example for this excerise, we can define it here. Else its recommended to use a json file.

#config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST", filter_dict={"tags": ["tool"]})
# Dont need filter as the model we are going to use models that support function calling.

config_list = [
  {
    "model": "gpt-4o",
    "api_type": "azure",
    "api_key": api_key,
    "base_url": api_base,
    "api_version": api_version
  },
  {
    "model": "gpt-35-turbo",
    "api_type": "azure",
    "api_key": api_key,
    "base_url": api_base,
    "api_version": api_version
  }
]

llm_config = {
    "config_list": config_list,
}

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

### Agent Definition 

In [9]:
# Agent that has access to only two functions. 

coder = autogen.AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. You have a dynamics365_call and a office365_call function, these functions can and should be used in parallel. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

# an agent that is intended to be used as a code executor.

os.makedirs("coding", exist_ok=True)
# Use DockerCommandLineCodeExecutor if docker is available to run the generated code.
# Using docker is safer than running the generated code directly.
# code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
code_executor = LocalCommandLineCodeExecutor(work_dir="coding")

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    system_message="A proxy for the user for executing code.",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"executor": code_executor},
)

### Function Definitions 
1) A function to call dynamics 365
2) A function to call office 365


In [10]:
# define functions according to the function description
# An example async function registered using register_for_llm and register_for_execution decorators

# Dynamics 365 function

@user_proxy.register_for_execution()
@coder.register_for_llm(description="Get the details associated with the given account name from dynamics 365 using WebAPI.")
async def dynamics365_call(account_name: Annotated[str, "Account Details ."]) -> str:

    from dotenv import load_dotenv
    import requests
    # Get the token and call service principle for Dynamics 365
    load_dotenv('.env')
    tenant_id = os.getenv('dynamicsSP_tenant_id')
    client_id = os.getenv('dynamicsSP_client_id')
    client_secret = os.getenv('dynamicsSP_client_secret')
    resource = os.getenv('dynamicsSP_resource')

    # OAuth 2.0 token endpoint  
    token_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'  

    # Prepare the data for the token request  
    data = {  
        'grant_type': 'client_credentials',  
        'client_id': client_id,  
        'client_secret': client_secret,  
        'scope': f'{resource}.default'  
    } 

    # Request the token  
    response = requests.post(token_url, data=data)  
    response_data = response.json()  

    if 'access_token' in response_data:  
        access_token = response_data['access_token']  
        #print("Access token retrieved successfully.")  
    else:  
        print("Error retrieving access token:", response_data) 
    
        # Set the API endpoint for retrieving accounts  
    #api_url = f'{resource}/api/data/v9.0/accounts'  
    api_url = f'{resource}/api/data/v9.0/accounts?$select=name,accountid&$filter=contains(name,\'{account_name}\')'


    # Set the headers for the request  
    headers = {  
        'Authorization': f'Bearer {access_token}',  
        'Content-Type': 'application/json',  
        'OData-MaxVersion': '4.0',  
        'OData-Version': '4.0',  
    }  
    
    # Make the request to retrieve account details  
    response = requests.get(api_url, headers=headers)  
    #accounts = response.json()  
    
    return response.json()

In [13]:
@user_proxy.register_for_execution()
@coder.register_for_llm(description="Get the emails associated with the given user and account name from outlook365.")
async def outlook365_call(account_name: Annotated[str, "Emails Details ."]) -> str:

    import requests
    from dotenv import load_dotenv
    load_dotenv('.env')

    client_id = os.getenv('outlookSP_client_id')
    tenant_id = os.getenv('outlookSP_tenant_id')
    client_secret = os.getenv('outlookSP_client_secret')
    scope = os.getenv('outlookSP_scope')
 
    # Get the access token
    url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token'
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': scope
    }
    response = requests.post(url, data=data)
    access_token = response.json().get('access_token')
    
    # print(response)
    # print(access_token)

    #user_id = 'a user id'  # Replace with the actual user ID
    user_id = os.getenv('outlookSP_user_id')
    url = f"https://graph.microsoft.com/v1.0/users/{user_id}/mailFolders/inbox/messages"
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json'
    }
    params = {
        #'$search': 'Alpine',
        '$search': f'"{account_name}"',
        '$select': 'subject,body,receivedDateTime'
    }
    
    response = requests.get(url, headers=headers, params=params)
    #print(response.status_code)
    #print(response.json())
    return response.json()


c:\Users\pdhiman\Anaconda3\envs\gRagAutogen\lib\site-packages\autogen\agentchat\conversable_agent.py:2489: UserWarning: Function 'outlook365_call' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
c:\Users\pdhiman\Anaconda3\envs\gRagAutogen\lib\site-packages\autogen\agentchat\conversable_agent.py:2408: UserWarning: Function 'outlook365_call' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


### Initiate a simple chat for testing the above agents

In [14]:
with Cache.disk() as cache:
    await user_proxy.a_initiate_chat(  # noqa: F704
        coder,
        message="Find the details for account Alpine Ski House and associated emails .",
        cache=cache,
    )

user_proxy (to chatbot):

Find the details for account Alpine Ski House and associated emails .

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool call (call_W1E3MRsixS4KLUz9QbmW7F2R): dynamics365_call *****
Arguments: 
{"account_name": "Alpine Ski House"}
*********************************************************************************
***** Suggested tool call (call_OHMQcpZq8fBV9prvBlwZpnu6): outlook365_call *****
Arguments: 
{"account_name": "Alpine Ski House"}
********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION dynamics365_call...



>>>>>>>> EXECUTING ASYNC FUNCTION outlook365_call...
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool (call_W1E3MRsixS4KLUz9QbmW7F2R) *****
{"@odata.context": "https://orgd4e65928.crm.dynamics.com/api/data/v9.0/$metadata#accounts(name,accountid)", "value": [{"@odata.etag": "W/\"6379902\"", "name": "Alpine Ski House (sample)", "accountid": "46f4a2bf-e63e-ef11-8409-002248277599"}]}
**********************************************************************

--------------------------------------------------------------------------------
user_proxy (to chatbot):

***** Response from calling tool (call_OHMQcpZq8fBV9prvBlwZpnu6) *****
{"@odata.context": "https://graph.microsoft.com/v1.0/$metadata#users('0f3ba324-d6a4-477e-8fea-bc90750ad285')/mailFolders('inbox')/messages(subject,body,receivedDateTime)", "value": [{"@odata.etag": "W/\"CQAAABYAAAAW7kwrgry4QYwXzkQhOoIkAAANcca7\"", "id": "AAMkAGU5MWY0NWZjLThhNmYtNGYyNS1iOTZkLTQ0NWFkOTVlYmU2NgBGAAAAAAD4GSQPmfDbRb9

#### Lets make the responses well formatted by using a formating / markdown agent.

In [15]:
markdownagent = autogen.AssistantAgent(
    name="Markdown_agent",
    system_message="Respond in markdown only",
    llm_config=llm_config,
)

# Add a function for robust group chat termination


@user_proxy.register_for_execution()
@markdownagent.register_for_llm()
@coder.register_for_llm(description="terminate the group chat")
def terminate_group_chat(message: Annotated[str, "Message to be sent to the group chat."]) -> str:
    return f"[GROUPCHAT_TERMINATE] {message}"


groupchat = autogen.GroupChat(agents=[user_proxy, coder, markdownagent], messages=[], max_round=12)

llm_config_manager = llm_config.copy()
llm_config_manager.pop("functions", None)
llm_config_manager.pop("tools", None)

manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config_manager,
    is_termination_msg=lambda x: "GROUPCHAT_TERMINATE" in x.get("content", ""),
)

#### Initiating a agent conversation.

In [16]:
message = """
1) Find the details for account Alpine Ski House and associated emails.
2) Pretty print the result as md.
3) Based on the email and account information, what should be the sales strategy for upcoming call.
4) when 1,2 and 3 are done, terminate the group chat
"""

with Cache.disk() as cache:
    await user_proxy.a_initiate_chat(  # noqa: F704
        manager,
        message=message,
        cache=cache,
    )

user_proxy (to chat_manager):


1) Find the details for account Alpine Ski House and associated emails.
2) Pretty print the result as md.
3) Based on the email and account information, what should be the sales strategy for upcoming call.
4) when 1,2 and 3 are done, terminate the group chat


--------------------------------------------------------------------------------
chatbot (to chat_manager):

***** Suggested tool call (call_MLcMyYoTSYNTGfqgBmW2SgL1): dynamics365_call *****
Arguments: 
{"account_name": "Alpine Ski House"}
*********************************************************************************
***** Suggested tool call (call_1Hm5yAQ1sgzEBTjeM3h2uzmE): outlook365_call *****
Arguments: 
{"account_name": "Alpine Ski House"}
********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING ASYNC FUNCTION dynamics365_call...

>>>>>>>> EXECUTING ASYNC FUNCTION ou